k-cross validation

In [6]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



In [7]:
df=pd.read_csv(r'C:\Users\Abdulmuxtor\Downloads\placementdata_with_faculty.csv')

In [8]:
df.head()

,StudentID,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty
0,1,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced,Business Administration
1,2,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed,Computer Science
2,3,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced,Computer Science
3,4,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed,Civil Engineering
4,5,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed,Electrical Engineering


In [10]:
df.drop(columns=['StudentID'],inplace=True)

In [11]:
df.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty
0,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced,Business Administration
1,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed,Computer Science
2,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced,Computer Science
3,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed,Civil Engineering
4,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed,Electrical Engineering


data prerocessing

missing values

In [9]:
df.isnull().sum()

StudentID                    0
CGPA                         0
Internships                  0
Projects                     0
Workshops/Certifications     0
AptitudeTestScore            0
SoftSkillsRating             0
ExtracurricularActivities    0
PlacementTraining            0
SSC_Marks                    0
HSC_Marks                    0
PlacementStatus              0
faculty                      0
dtype: int64

In [15]:
for col in df.columns:
    if df[col].dtype=='object':
        df[col].fillna(df[col].mode()[0],inplace=True)
    else:
        df[col].fillna(df[col].mode(),inplace=True)

C:\Users\Abdulmuxtor\AppData\Local\Temp\ipykernel_2912\4165299538.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode(),inplace=True)
C:\Users\Abdulmuxtor\AppData\Local\Temp\ipykernel_2912\4165299538.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

encoding

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CGPA                       10000 non-null  float64
 1   Internships                10000 non-null  int64  
 2   Projects                   10000 non-null  int64  
 3   Workshops/Certifications   10000 non-null  int64  
 4   AptitudeTestScore          10000 non-null  int64  
 5   SoftSkillsRating           10000 non-null  float64
 6   ExtracurricularActivities  10000 non-null  object 
 7   PlacementTraining          10000 non-null  object 
 8   SSC_Marks                  10000 non-null  int64  
 9   HSC_Marks                  10000 non-null  int64  
 10  PlacementStatus            10000 non-null  object 
 11  faculty                    10000 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 937.6+ KB


In [ ]:
df['CGPA'].value_counts() # bitta ustundagi nunique elementlarni chiqarib beradi 

CGPA
6.5    833
7.6    814
7.5    798
8.3    662
8.2    655
7.7    622
8.4    608
7.4    590
8.1    534
7.8    453
8.0    445
8.5    428
7.9    384
8.6    309
6.6    278
7.3    267
6.7    265
6.8    212
8.7    187
6.9    165
7.2    127
7.0    121
7.1     92
8.8     86
8.9     49
9.0     13
9.1      3
Name: count, dtype: int64

In [ ]:
cardinality=df.nunique() # umumiy har bir ustundagi nunique elementlarni chiqarib beradi

In [20]:
cardinality

CGPA                         27
Internships                   3
Projects                      4
Workshops/Certifications      4
AptitudeTestScore            31
SoftSkillsRating             19
ExtracurricularActivities     2
PlacementTraining             2
SSC_Marks                    36
HSC_Marks                    32
PlacementStatus               2
faculty                       6
dtype: int64

In [23]:
df_encoded=df.copy()

In [26]:
for col in df_encoded.select_dtypes(include=['object']).columns:
    cardinality=df_encoded[col].nunique()
    if cardinality<=5:
        le=LabelEncoder()
        df_encoded[col]=le.fit_transform(df_encoded[col])
    else:
        df_encoded=pd.get_dummies(df_encoded,columns=[col],dtype=int,drop_first=True)
df=df_encoded

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CGPA                             10000 non-null  float64
 1   Internships                      10000 non-null  int64  
 2   Projects                         10000 non-null  int64  
 3   Workshops/Certifications         10000 non-null  int64  
 4   AptitudeTestScore                10000 non-null  int64  
 5   SoftSkillsRating                 10000 non-null  float64
 6   ExtracurricularActivities        10000 non-null  int64  
 7   PlacementTraining                10000 non-null  int64  
 8   SSC_Marks                        10000 non-null  int64  
 9   HSC_Marks                        10000 non-null  int64  
 10  PlacementStatus                  10000 non-null  int64  
 11  faculty_Business Administration  10000 non-null  int64  
 12  faculty_Civil Engin